In [1]:
import hashlib
import sqlite3
import os

from pathlib import Path

import pandas as pd
from typing import List, Optional
from sqlmodel import SQLModel, create_engine, Session, Field, select

from typing import Optional

from sqlite_db_schema import RAG_Create
from database import engine

In [2]:

def list_files_in_directory(directory: str) -> List[str]:
    # Create a Path object from the directory
    dir_path = Path(directory)
    
    # Check if the provided directory exists and is a directory
    if not dir_path.is_dir():
        raise ValueError(f"The provided path '{directory}' is not a valid directory.")

    # List all files in the directory (non-recursive)
    file_list = [str(file.name) for file in dir_path.iterdir() if file.is_file()]

    return file_list

# Example usage:
# files = list_files_in_directory("/path/to/your/directory")
# print(files)

In [3]:
# Function to calculate file hash
def file_hash(filename):
    hash_sha256 = hashlib.sha256()

    with open(filename, "rb") as file:
        for chunk in iter(lambda: file.read(4096), b""):
            hash_sha256.update(chunk)

    return hash_sha256.hexdigest()

# Function to add hash to database
def add_hash_to_db(filename):
    # Calculate the file hash
    hash_value = file_hash(filename)

    # Create database engine and session
    #engine = create_db_and_table()
    
    with Session(engine) as session:
        # Check if the file already exists in the database
        statement = select(RAG_Create).where(RAG_Create.filename == filename)
        result = session.exec(statement).first()
        
        if result:
            print(f"File '{filename}' is already in the database.")
            return result.hash

        # Create a new FileHash record
        new_file_hash = RAG_Create(filename=filename, hash=hash_value)
        
        # Add and commit to the database
        session.add(new_file_hash)
        session.commit()

        print(f"File '{filename}' added to the database with hash '{hash_value}'.")
        return hash_value

# Example usage:
# hash_value = add_hash_to_db("example.txt")
# print(f"The SHA-256 hash of the file is: {hash_value}")

In [4]:
list_files = list_files_in_directory(Path("images"))

In [6]:
from pathlib import Path
from typing import List

def list_file_objects_in_directory(directory: str) -> List[Path]:
    # Create a Path object from the directory
    dir_path = Path(directory)
    
    # Check if the provided directory exists and is a directory
    if not dir_path.is_dir():
        raise ValueError(f"The provided path '{directory}' is not a valid directory.")

    # List all file objects in the directory (non-recursive)
    file_objects = [file for file in dir_path.iterdir() if file.is_file()]

    return file_objects

In [5]:
for file in list_files:
    add_hash_to_db(file)

FileNotFoundError: [Errno 2] No such file or directory: 'el_nino.html'

In [ ]:
import hashlib

def file_hash(filename):
    # Create a new sha256 hash object
    hash_sha256 = hashlib.sha256()

    # Open the file in binary mode and read chunks
    with open(filename, "rb") as file:
        for chunk in iter(lambda: file.read(4096), b""):
            hash_sha256.update(chunk)

    # Return the hexadecimal digest of the hash
    return hash_sha256.hexdigest()

# Example usage:
# hash_value = file_hash("example.txt")
# print(f"The SHA-256 hash of the file is: {hash_value}")

In [ ]:
file_hash("hash_file.ipynb")

In [ ]:

def generate_file_hash(file_obj, hash_algo="sha256", chunk_size=8192):
    """
    Generate a unique hash for a file based on its content.
    
    Parameters:
    - file_obj: File object opened in binary mode.
    - hash_algo: Hashing algorithm to use (default is 'sha256').
    - chunk_size: Size of the chunks to read from the file at a time (default is 8192 bytes).
    
    Returns:
    - A string representing the unique hash of the file content.
    """
    # Initialize the hash object with the chosen algorithm
    hash_func = hashlib.new(hash_algo)
    
    # Read the file in chunks to avoid using too much memory with large files
    for chunk in iter(lambda: file_obj.read(chunk_size), b""):
        hash_func.update(chunk)
    
    #Create a db session
    session = Session(engine)
    hash = hash_func.hexdigest()
    data_base_entry = RAG_Create(filename="text.pdf",hash=hash )

    session.add(data_base_entry)
    
    # Return the hexadecimal representation of the hash
    
    
    #return hash_func.hexdigest()


In [ ]:
generate_file_hash

In [ ]:
# Example usage:
with open("./images/CoT.pdf", "rb") as f:
    file_hash = generate_file_hash(f)
    print(f"The file's unique hash is: {file_hash}")

In [ ]:
cursor